Mount Drive

In [ ]:
from google.colab import drive
from google.colab import files
print("Getting Data from Google drive")
drive.mount('/content/drive/', force_remount ="True")
#text_en = open("/content/drive/MyDrive/BTP/kddcup.data.csv","r").read()


Getting Data from Google drive
Mounted at /content/drive/


Get *data* and Preprocessing

In [ ]:
import numpy as nm  
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as mtp  
import pandas as pd  
data_set= pd.read_csv('/content/drive/MyDrive/BTP/kddcup.data.csv') 
#data_Set = data_set.dropna('columns')
x= data_set.iloc[:, 0:41].values 
y= data_set.iloc[:, 41].values  
d = {}
# Counting data poihnts of each class
for val in y:
  if val in d:
    d[val] = d[val]+1
  else:
    d[val] = 1

print(d)

from sklearn.preprocessing import LabelEncoder  
label_encoder_x1= LabelEncoder()  
label_encoder_x2= LabelEncoder() 
label_encoder_x3= LabelEncoder()  
    
x[:, 1]= label_encoder_x1.fit_transform(x[:, 1])  
x[:, 2]= label_encoder_x2.fit_transform(x[:, 2])  
x[:, 3]= label_encoder_x3.fit_transform(x[:, 3]) 

attack_class = {'normal.': 'Normal', 'back.': 'DOS', 'smurf.':'DOS', 'neptune.':'DOS', 'pod.': 'DOS', 'teardrop.':'DOS', 'land.': 'DOS',
                'multihop.':'R2L', 'ftp_write.': 'R2L', 'guess_passwd.': 'R2L', 'imap.': 'R2L', 'phf.': 'R2L', 'spy.':'R2L', 'warezmaster.':'R2L', 'warezclient.':'R2L',
                'rootkit.':'U2R', 'loadmodule.':'U2R', 'buffer_overflow.': 'U2R', 'perl.': 'U2R',
                'portsweep.':'PROBE', 'satan.':'PROBE', 'ipsweep.': 'PROBE', 'nmap.':'PROBE'}

from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.10, random_state=0) 



for i in range(len(y_train)):
  y_train[i] = attack_class[y_train[i]]

for i in range(len(y_test)):
  y_test[i] = attack_class[y_test[i]]


y_train2 = y_train.copy()
y_test2 = y_test.copy()
for i in range(len(y_train2)):
  if y_train2[i] == 'Normal':
    y_train2[i] = 'Normal'
  else:
    y_train2[i] = 'Intrusion'

for i in range(len(y_test2)):
  if y_test2[i] == 'Normal':
    y_test2[i] = 'Normal'
  else:
    y_test2[i] = 'Intrusion'

{'normal.': 972780, 'buffer_overflow.': 30, 'loadmodule.': 9, 'perl.': 3, 'neptune.': 1072017, 'smurf.': 2807886, 'guess_passwd.': 53, 'pod.': 264, 'teardrop.': 979, 'portsweep.': 10413, 'ipsweep.': 12481, 'land.': 21, 'ftp_write.': 8, 'back.': 2203, 'imap.': 12, 'satan.': 15892, 'phf.': 4, 'nmap.': 2316, 'multihop.': 7, 'warezmaster.': 20, 'warezclient.': 1020, 'spy.': 2, 'rootkit.': 10}


In [ ]:
print(len(y))

4898430


In [ ]:
import time
from sklearn import metrics

## **K-Means**

In [ ]:
from sklearn.cluster import KMeans
kmeans1 = KMeans(n_clusters = 5)
time_start = time.time()
kmeans1.fit(x_train)
time_end = time.time()
print("Training time: ", time_end - time_start)

time_start = time.time()
y_pred = kmeans1.predict(x_test)
time_end = time.time()
print("Testing time: ", time_end - time_start)
print(y_pred)

for i in range(5):
  print(list(y_pred).count(i))

print(nm.unique(y_pred))

Training time:  67.74123787879944
Testing time:  1.8915979862213135
[0 0 0 ... 0 0 0]
489842
0
0
0
1
[0 4]


In [ ]:
from sklearn.cluster import KMeans

kmeans2 = KMeans(n_clusters = 2)
time_start = time.time()
kmeans2.fit(x_train)
time_end = time.time()

print("Training time: ", time_end - time_start)
time_start = time.time()
y_pred2 = kmeans2.predict(x_train)
time_end = time.time()
print("Testing time: ", time_end - time_start)
print(y_pred2)

for i in range(2):
  print(list(y_pred2).count(i))


print(nm.unique(y_pred2))

Training time:  49.565539836883545
Testing time:  15.658473491668701
[0 0 0 ... 0 0 0]
4408583
4
[0 1]


## **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

classifier1 = RandomForestClassifier(n_estimators = 50)
time_start = time.time()
classifier1.fit(x_train, y_train)
time_end = time.time()
print("Training time: ", time_end - time_start)

time_start = time.time()
y_pred = classifier1.predict(x_test)
time_end = time.time()
print("Testing time: ", time_end - time_start)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
result1 = classification_report(y_test, y_pred, zero_division = 0)
print(result1)

Training time:  536.7322769165039
Testing time:  4.4007720947265625
Accuracy: 0.9999714194139755
[[388263      2      0      0      0]
 [     0  97321      0      0      0]
 [     0      3   4140      0      0]
 [     0      4      0    104      0]
 [     0      5      0      0      1]]
              precision    recall  f1-score   support

         DOS       1.00      1.00      1.00    388265
      Normal       1.00      1.00      1.00     97321
       PROBE       1.00      1.00      1.00      4143
         R2L       1.00      0.96      0.98       108
         U2R       1.00      0.17      0.29         6

    accuracy                           1.00    489843
   macro avg       1.00      0.83      0.85    489843
weighted avg       1.00      1.00      1.00    489843



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

classifier2 = RandomForestClassifier(n_estimators = 50)
time_start = time.time()
classifier2.fit(x_train, y_train2)
time_end = time.time()
print("Training time: ", time_end - time_start)

time_start = time.time()
y_pred2 = classifier2.predict(x_test)
time_end = time.time()
print("Testing time: ", time_end - time_start)
print("Accuracy:",metrics.accuracy_score(y_test2, y_pred2))
print(confusion_matrix(y_test2, y_pred2))
result2 = classification_report(y_test2, y_pred2)
print(result2)

Training time:  478.39593982696533
Testing time:  3.7377870082855225
Accuracy: 0.9999714194139755
[[392509     13]
 [     1  97320]]
              precision    recall  f1-score   support

   Intrusion       1.00      1.00      1.00    392522
      Normal       1.00      1.00      1.00     97321

    accuracy                           1.00    489843
   macro avg       1.00      1.00      1.00    489843
weighted avg       1.00      1.00      1.00    489843



## **Naive- Bayes Classifier**

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb1 = GaussianNB()
time_start = time.time()
gnb1.fit(x_train, y_train)
time_end   = time.time()
print("Training time: ", time_end - time_start)

time_start = time.time()
y_pred = gnb1.predict(x_test)
time_end = time.time()
print("Testing time: ", time_end - time_start)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
result1 = classification_report(y_test, y_pred, zero_division =  0)
print(confusion_matrix(y_test, y_pred))
print(result1)


Training time:  26.872153997421265
Testing time:  2.535879373550415
Accuracy: 0.9281075773257962
[[387935    262      0      0     68]
 [ 27294  66590    787     98   2552]
 [  2626   1158     98      0    261]
 [     5     43     19      0     41]
 [     1      1      0      0      4]]
              precision    recall  f1-score   support

         DOS       0.93      1.00      0.96    388265
      Normal       0.98      0.68      0.81     97321
       PROBE       0.11      0.02      0.04      4143
         R2L       0.00      0.00      0.00       108
         U2R       0.00      0.67      0.00         6

    accuracy                           0.93    489843
   macro avg       0.40      0.47      0.36    489843
weighted avg       0.93      0.93      0.92    489843



In [ ]:
from sklearn.metrics import confusion_matrix
print(nm.unique(y_pred))
print(confusion_matrix(y_test, y_pred))

['DOS' 'Normal' 'PROBE' 'R2L' 'U2R']
[[387935    262      0      0     68]
 [ 27294  66590    787     98   2552]
 [  2626   1158     98      0    261]
 [     5     43     19      0     41]
 [     1      1      0      0      4]]


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report


gnb2 = GaussianNB()
time_start = time.time()
gnb2.fit(x_train, y_train2)
time_end = time.time()
print("Training time: ", time_end - time_start)

time_start = time.time()
y_pred2 = gnb2.predict(x_test)
time_end = time.time()
print("Testing time: ", time_end - time_start)
print("Accuracy:",metrics.accuracy_score(y_test2, y_pred2))
result2 = classification_report(y_test2, y_pred2)
print(confusion_matrix(y_test2, y_pred2))
print(result2)

Training time:  25.112592935562134
Testing time:  2.0298562049865723
Accuracy: 0.905853508164861
[[347174  45348]
 [   769  96552]]
              precision    recall  f1-score   support

   Intrusion       1.00      0.88      0.94    392522
      Normal       0.68      0.99      0.81     97321

    accuracy                           0.91    489843
   macro avg       0.84      0.94      0.87    489843
weighted avg       0.93      0.91      0.91    489843



## **Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

clfd = DecisionTreeClassifier(criterion ="entropy", max_depth = 4)
time_start = time.time()
clfd.fit(x_train, y_train)
time_end = time.time()
print("Training time: ", time_end - time_start)

time_start = time.time()
y_pred = clfd.predict(x_test)
time_end = time.time()
print("Testing time: ", time_end- time_start)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
result = classification_report(y_test, y_pred, zero_division = 0)
print(confusion_matrix(y_test, y_pred))
print(result)


Training time:  51.00517392158508
Testing time:  1.696615219116211
Accuracy: 0.9969684164109724
[[388031    230      4      0      0]
 [     3  96483    835      0      0]
 [    26    273   3844      0      0]
 [     0    106      2      0      0]
 [     0      4      2      0      0]]
              precision    recall  f1-score   support

         DOS       1.00      1.00      1.00    388265
      Normal       0.99      0.99      0.99     97321
       PROBE       0.82      0.93      0.87      4143
         R2L       0.00      0.00      0.00       108
         U2R       0.00      0.00      0.00         6

    accuracy                           1.00    489843
   macro avg       0.56      0.58      0.57    489843
weighted avg       1.00      1.00      1.00    489843



In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

clfd = DecisionTreeClassifier(criterion ="entropy", max_depth = 4)
time_start = time.time()
clfd.fit(x_train, y_train2)
time_end = time.time()
print("Training time: ", time_end - time_start)

time_start = time.time()
y_pred2 = clfd.predict(x_test)
time_end = time.time()
print("Testing time: ", time_end - time_start)
print("Accuracy:",metrics.accuracy_score(y_test2, y_pred2))
result = classification_report(y_test2, y_pred2, zero_division = 0)
print(confusion_matrix(y_test2, y_pred2))
print(result)


Training time:  49.56501889228821
Testing time:  1.7028801441192627
Accuracy: 0.9988731083224625
[[391982    540]
 [    12  97309]]
              precision    recall  f1-score   support

   Intrusion       1.00      1.00      1.00    392522
      Normal       0.99      1.00      1.00     97321

    accuracy                           1.00    489843
   macro avg       1.00      1.00      1.00    489843
weighted avg       1.00      1.00      1.00    489843

